In [1]:
import sys

# sys.path.append(r"/Volumes/funkflower/Users/Yingyue/Gates_Lab_Suite-master")
sys.path.append(r"/Users/yzhu/yzhu_work/gates projects")
from Core_Definition import *
from Auto_Algorithm import *
from Visualization import *
import os
from SPAM import *
import numpy as np
import xlsxwriter as xlsx
from scipy import optimize
import random
import math
from math import floor
import uuid
import itertools
sys.path.append(r"/Users/yzhu/yzhu_work/gates projects/EAB")
from EAB_CB_data_analysis import *

In [2]:
import json, pickle
import qiskit
#transpiler reducinig qiskit circuit to QASM strings
from qiskit import transpile
# Use AerSimulator
from qiskit_aer import AerSimulator
from qiskit import IBMQ, QuantumCircuit, execute
# from qiskit.providers.ibmq.managed import IBMQJobManager
# from qiskit.extensions import UnitaryGate
from qiskit.quantum_info import Pauli, Clifford, Statevector,Operator
from qiskit.visualization import array_to_latex
from scipy.stats import sem, unitary_group
from scipy.linalg import sqrtm,expm
import qiskit.quantum_info as qi
from qiskit.quantum_info import Pauli
from qiskit_aer.noise import NoiseModel, pauli_error, amplitude_damping_error, ReadoutError
import pylatexenc
from qiskit_ionq import IonQProvider

# functions

function for gates

In [3]:
#prepare eigenstate of each pauli
def prepare_pauli_eigenstate_gates(qc,pauli,q_index):
    if (pauli=="I"):
        pauli = random.choice(['X','Y','Z'])
    if (pauli=="X"):
        qc.Add_Gate(Quantum_Gate("RY",q_index,angle=2))
    elif (pauli=="Y"):
        qc.Add_Gate(Quantum_Gate("RX",q_index,angle=-2))
    elif (pauli=="Z"):
        pass

        
        
        
#should we set_mapping in this function? need to think about if it takes mapping into account   
def add_pauli_twirl(qc,n):
    pauliLayer = [random.choice(['I','X','Y','Z']) for j in range(n)]
    q_index=0
    for pauli in pauliLayer :
        if (pauli=="I"):
            pass
        elif (pauli=="X"):
            qc.Add_Gate(Quantum_Gate("RX",q_index,angle=1))
        elif (pauli=="Y"):
            qc.Add_Gate(Quantum_Gate("RY",q_index,angle=1))
        elif (pauli=="Z"):
            qc.Add_Gate(Quantum_Gate("AZ",q_index,angle=1))
        q_index+=1
    return pauliLayer
    
def add_clifford_layer(qc,n,clifford):
    if (clifford=="CNOT"):
        for i in range (floor(n/2)):
            qc.Add_Gate(Quantum_Gate("CNOT",2*i,2*i+1))
    if (clifford=="XX"):
        for i in range (floor(n/2)):
            qc.Add_Gate(Quantum_Gate("FTXA",2*i,2*i+1,angle=1/4))

def measure_pauli_1q_gates(qc,index,pauli=None):
    if pauli == 'I' or pauli == 'Z':
        pass
    elif pauli == 'X':
        qc.Add_Gate(Quantum_Gate("RY",index,angle=-2))
    elif pauli == 'Y':
        qc.Add_Gate(Quantum_Gate("RX",index,angle=2))
    else:
        assert 1==0            


funtion for qiskit

In [4]:
#for qiskit

def prepare_pauli_eigenstate_1q(circuit,index,pauli=None):
	if pauli == 'I':
		pauli = random.choice(['X','Y','Z']) 
	# note: For Pauli I, prepare an arbitrary eigenstate.
	if pauli == 'Z':
		circuit.id([index])
	elif pauli == 'X':
		circuit.h([index])
	elif pauli == 'Y':
		circuit.h([index])
		circuit.s([index])
	else:
		print(pauli)
		assert 1==0

def pauli_gate_1q(circuit,index,pauli=None): #For stabilizer simulator to work, cannot use Pauli class
	if pauli == 'I':
		circuit.id([index])
	elif pauli == 'Z':
		circuit.z([index])
	elif pauli == 'X':
		circuit.x([index])
	elif pauli == 'Y':
		circuit.y([index])
	else:
		assert 1==0

def add_XX_clifford(circ,index):
    circ.z(2*index)
    circ.h(2*index)
    circ.x(2*index)
    circ.z(2*index)
    circ.cx(2*index,2*index+1)
    circ.s(2*index)
    circ.h(2*index)
    circ.x(2*index)
    circ.h(2*index+1)
    circ.s(2*index+1)
    circ.h(2*index+1)
        
def construct_measurement_qiskit(pauli_sample,qc_gates):
    pauliOp = Pauli(''.join(pauli_sample))
    pauliOp = pauliOp.evolve(Clifford(qc_gates).adjoint()) 
    measurement_setting = pauliOp.to_label()
# there could be a '-' in the Pauli label
    if measurement_setting[0].isupper() is False:
        measurement_setting = measurement_setting[1:]
    measurement_setting = measurement_setting[::-1]
    return measurement_setting
    #print(measurement_setting)

def measure_pauli_1q(circuit,index,pauli=None):
	if pauli == 'I' or pauli == 'Z':
		circuit.id([index])
	elif pauli == 'X':
		circuit.h([index])
	elif pauli == 'Y':
		circuit.s([index])
		circuit.s([index])
		circuit.s([index])
		circuit.h([index])
	else:
		assert 1==0

In [5]:
## Additional functions

def int_to_pauli(i,n):
    p = np.base_repr(i,base=4)
    p = '0'*(n-len(p)) + p
    p = p.replace('0','I').replace('1', 'X').replace('2', 'Y').replace('3', 'Z')
    return p

def commute(p,q):
    c = 1
    n = len(p)
    for i in range(n):
        if p[i] != 'I' and q[i] != 'I':
            if p[i] != q[i]:
                c *= -1
    return c

def fidelity_to_error(pauli_fidelity,n):
    N = 4**n
    pauli_error = {}
    for i in range(N):
        p = int_to_pauli(i,n)
        pauli_error[p] = 0
        for j in range(N):
            q = int_to_pauli(j,n)
            pauli_error[p] += pauli_fidelity[q] * commute(p,q) / N
    return pauli_error
    

In [6]:
os.environ["IONQ_API_KEY"] = "2IrrtuzJntPCcKvJzkKczpA1TJKTH9LO"
my_api_key = os.getenv("IONQ_API_KEY")
# print (my_api_key)
provider = IonQProvider(my_api_key)

backend = provider.get_backend("ionq_simulator")

In [7]:

n=2
# ancillae=2
S=Quantum_Circuit(n,"qc")
S_explicit=Quantum_Circuit(n,"qc_e")

depth=[2**x for x in range(1,6)] #depth for Concatenation
C=20 #number of samples per depth

# all_circuits=[]#a list of lists. circuits for all depths. each element is a list of circuit of a certain depth d.
all_circuits_explicit={}
# circuits=[] #a list of circuits with a certain depth d
circuits_explicit=[]
# a list of circuits at all depth d for a certain pauli
circuits_explicit_pauli=[]
cb_data={} # the object that will be saved eventually
results={}
result_list=[]
clifford_layer="XX"

#for qiskit 
# noise_model = NoiseModel()
# eps=0.005
# noise_model.add_all_qubit_quantum_error(pauli_error([('II',(1-eps)**2),('XX',eps**2),('IX',eps*(1-eps)),('XI',eps*(1-eps))]),['cx'])
# backend = AerSimulator()
# backend = AerSimulator(method='stabilizer')
shots=2000

pauli_sample_list = [''.join(s) for s in itertools.product(['X','Y','Z'], repeat = n)]
for pauli_sample in pauli_sample_list:
    print ("\npauli label:",pauli_sample)
    all_circuits_explicit[pauli_sample]=[]
    cb_data[pauli_sample]={}
    cb_data[pauli_sample]["parameters"]={}
    cb_data[pauli_sample]["parameters"]['n'] = n 
    cb_data[pauli_sample]["parameters"]['shots'] = shots 
    cb_data[pauli_sample]["parameters"]['Lrange'] = depth
    cb_data[pauli_sample]["parameters"]['C'] = C
#     cb_data["parameters"]['repeat'] = repeat
    cb_data[pauli_sample]["result"]=[]
    for d in depth:

        print ("\nDepth = %d"%d)
        print ("Number of samples = %d"%C)
        for i in range (C):
            
            #qiskit_circ=QuantumCircuit(2*nqubit,2*nqubit)
            qiskit_c_state = QuantumCircuit(n,n)
            qiskit_c_gates = QuantumCircuit(n)


            for j in range(n):
                prepare_pauli_eigenstate_1q(qiskit_c_state,j,pauli=pauli_sample[n-j-1])
            qiskit_c_state.barrier()


            for j in range(d):
                #gates circuit add pauli layer
#                 layer=add_pauli_twirl(S,n)
                #gates explicit cnot circuit add pauli layer
                layer=add_pauli_twirl(S_explicit,n)

                #qiskit circuit add pauli layer
                for q in range(n):
                    pauli_gate_1q(qiskit_c_gates,q,pauli=layer[q])

                if clifford_layer == 'Id':
                    pass
                elif clifford_layer == 'CNOT':
                    ngates = int(n/2)
                    for q in range(ngates):
                        qiskit_c_gates.cx(2*q,2*q+1)
                elif clifford_layer == 'XX':
                    ngates = int(n/2)
                    for q in range(ngates):
                        add_XX_clifford(qiskit_c_gates,q)
                qiskit_c_gates.barrier()
            

            layer=add_pauli_twirl(S_explicit,n)
            #qiskit circuit add Pauli twirl
            for j in range(n):
                pauli_gate_1q(qiskit_c_gates,j,pauli=layer[j])

            #save the stablizer representation
            cliffordOp = Clifford(qiskit_c_gates)
        
            #construct measurement basis
            measurement_basis=construct_measurement_qiskit(pauli_sample,qiskit_c_gates)
        
            #for qiskit: append gates to state
            qiskit_circ = qiskit_c_state.compose(qiskit_c_gates,range(n))

            #qiskit measurement
            qiskit_circ.barrier()
            for j in range(n):
                measure_pauli_1q(qiskit_circ,j,pauli=measurement_basis[j])

            qiskit_circ.barrier()
            qiskit_circ.measure([j for j in range(n)], [j for j in range(n)])


            #for qiskit 
            # qc_compiled = transpile(qiskit_circ, backend)
            job_sim = backend.run(qiskit_circ, shots=2000, noise_model="harmony") 
            # job_sim = backend.run(qc_compiled, shots=2000, noise_model="harmony") 
            result_sim = job_sim.result()
            counts_qiskit = result_sim.get_counts(qiskit_circ)


            

            results["L"]=d
            results["circuit"] = qiskit_circ.qasm
            results["clifford"] = cliffordOp.to_dict()
            results["clifford_layer"] = clifford_layer
            results["pauli"] = measurement_basis
            results["counts"]=counts_qiskit 

            cb_data[pauli_sample]["result"].append(results)
            results={}



pauli_sample_list = [''.join(s) for s in itertools.product(['X','Y','Z'], repeat = n)]
cb_data["n"] = n
cb_data["pauli_sample_list"] = pauli_sample_list    


pauli label: XX

Depth = 2
Number of samples = 20

Depth = 4
Number of samples = 20

Depth = 8
Number of samples = 20

Depth = 16
Number of samples = 20

Depth = 32
Number of samples = 20

pauli label: XY

Depth = 2
Number of samples = 20


In [ ]:
# print (all_circuits_explicit["XX"][40])

In [30]:
print (cb_data)

{'XX': {'parameters': {'n': 2, 'shots': 2000, 'Lrange': [2, 4, 8, 16, 32], 'C': 20}, 'result': [{'L': 2, 'circuit': <bound method QuantumCircuit.qasm of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x14182c220>>, 'clifford': {'stabilizer': ['+IZ', '+ZI'], 'destabilizer': ['-IX', '+XI']}, 'clifford_layer': 'XX', 'pauli': 'XX', 'counts': {'01': 2000}}, {'L': 2, 'circuit': <bound method QuantumCircuit.qasm of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x1404ac250>>, 'clifford': {'stabilizer': ['+IZ', '+ZI'], 'destabilizer': ['-IX', '+XI']}, 'clifford_layer': 'XX', 'pauli': 'XX', 'counts': {'01': 2000}}, {'L': 2, 'circuit': <bound method QuantumCircuit.qasm of <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x1419c24f0>>, 'clifford': {'stabilizer': ['+IZ', '-ZI'], 'destabilizer': ['+IX', '-XI']}, 'clifford_layer': 'XX', 'pauli': 'XX', 'counts': {'10': 2000}}, {'L': 2, 'circuit': <bound method QuantumCircuit.qasm of <qiskit.circuit.quantumcircuit.QuantumCircu

In [27]:
pauli_sample_list = [''.join(s) for s in itertools.product(['X','Y','Z'], repeat = n)]
cb_data["n"] = n
cb_data["pauli_sample_list"] = pauli_sample_list 

filename = 'sim_cb_20240422_FTXX45'
token = ''.join(random.choice([str(j) for j in range(10)]) for i in range(10))
filename += '_' + token
# pathname="/Volumes/funkflower/Users/Yingyue/Gates_Lab_Suite-master/PauliNoiseEstimation/circuits/04222024/"
# with open(pathname + filename, 'wb') as outfile:
#     pickle.dump(cb_data, outfile)

In [17]:
print (filename)

sim_cb_20240422_FTXX45_9361635663


In [18]:
# print (all_circuits_explicit[0])
# for pauli_sample in pauli_sample_list:
#     for i in range(len(depth)):
#         cricfname=pauli_sample+"_d="+str(depth[i])+".txt"
#         with open(pathname+cricfname, "w") as text_file:
#             for line in all_circuits_explicit[pauli_sample][i]:
#                 text_file.write(line+"\n")


FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/funkflower/Users/Yingyue/Gates_Lab_Suite-master/PauliNoiseEstimation/circuits/04222024/XX_d=2.txt'

In [28]:
import sys
sys.path.append(r"/Users/yzhu/yzhu_work/gates projects/EAB/")
import CB_process_modified
import pdb
'''Use these if read from file'''
# token = "8799418287"
# filename = "sim_cb_cnot_2023sep_9167558962"
# with open(pathname + filename, 'rb') as infile:
#     cb_data = pickle.load(infile)

data=cb_data

n = data["n"]
pauli_sample_list = data["pauli_sample_list"]
'''Specify a set of Pauli you want to estimate'''
pauli_request_list = [''.join(s) for s in itertools.product(['I','X','Y','Z'], repeat = n)] #full

fidelity_list = {} 
stdev_list = {}

for pauli_sample in pauli_sample_list:
    cb_data_temp = data[pauli_sample]

    # n = cb_data["parameters"]['n']
    # n_total = cb_data["parameters"]['n_total'] 
    shots = cb_data_temp ["parameters"]['shots'] 
    Lrange = cb_data_temp ["parameters"]['Lrange']
    C = cb_data_temp ["parameters"]['C'] 
#     eps_readout = cb_data_temp ["parameters"]['eps_readout'] 
#     repeat = cb_data_temp ["parameters"]['repeat']
#     pdb.set_trace()
    cb_result = CB_process_modified.process_CB(n, C, shots, 1, Lrange, cb_data_temp , pauli_sample = pauli_sample, periodic=True,use_density_matrix=False, intercept_cb=False)
    raw_fidelity_list = cb_result["fidelity_list"]
    
    new_sub_label = []

    for sub_label in raw_fidelity_list.keys():
        if sub_label in fidelity_list:
            continue # wasteful!
        elif(sub_label == 'I'*n):
            fidelity_list[sub_label] = 1.0
            stdev_list[sub_label] = 0.0
        else:
            alpha, alpha_err = CB_process_modified.fit_CB(Lrange, raw_fidelity_list[sub_label])
            fidelity_list[sub_label] = alpha
            stdev_list[sub_label] = alpha_err
            new_sub_label.append(sub_label)

    print("CB setting: ",pauli_sample[::-1]," Pauli fidelities calculated: ", [sub_label[::-1] for sub_label in new_sub_label])


# print(fidelity_list)

# print(stdev_list)
print("Parameters: n = %d, C = %d, " % (n,C), "L = ", str(Lrange))



# Average fidelity
print("Total error = ", 1-np.mean(list(fidelity_list.values())))

print("Label / Pauli infidelity / Standard deviation")
for pauli_label in pauli_request_list:
    print(pauli_label[::-1], 1-fidelity_list[pauli_label], stdev_list[pauli_label])

/Users/yzhu/anaconda3/envs/snowflakes/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:803: RuntimeWarning: divide by zero encountered in divide
  transform = 1.0 / sigma
/Users/yzhu/anaconda3/envs/snowflakes/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:488: RuntimeWarning: invalid value encountered in multiply
  return transform * (func(xdata, *params) - ydata)
/Users/yzhu/anaconda3/envs/snowflakes/lib/python3.8/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


CB setting:  XX  Pauli fidelities calculated:  ['XI', 'IX', 'XX']
CB setting:  YX  Pauli fidelities calculated:  ['YI', 'YX']
CB setting:  ZX  Pauli fidelities calculated:  ['ZI', 'ZX']
CB setting:  XY  Pauli fidelities calculated:  ['IY', 'XY']
CB setting:  YY  Pauli fidelities calculated:  ['YY']
CB setting:  ZY  Pauli fidelities calculated:  ['ZY']
CB setting:  XZ  Pauli fidelities calculated:  ['IZ', 'XZ']
CB setting:  YZ  Pauli fidelities calculated:  ['YZ']
CB setting:  ZZ  Pauli fidelities calculated:  ['ZZ']
Parameters: n = 2, C = 20,  L =  [2, 4, 8, 16, 32]
Total error =  0.0
Label / Pauli infidelity / Standard deviation
II 0.0 0.0
XI 0.0 inf
YI 0.0 inf
ZI 0.0 inf
IX 0.0 inf
XX 0.0 inf
YX 0.0 inf
ZX 0.0 inf
IY 0.0 inf
XY 0.0 inf
YY 0.0 inf
ZY 0.0 inf
IZ 0.0 inf
XZ 0.0 inf
YZ 0.0 inf
ZZ 0.0 inf


In [29]:
error_list = fidelity_to_error(fidelity_list,n)
print("Label / Pauli error rates")
for pauli_label in pauli_request_list:
    print(pauli_label[::-1], error_list[pauli_label])

Label / Pauli error rates
II 1.0
XI 0.0
YI 0.0
ZI 0.0
IX 0.0
XX 0.0
YX 0.0
ZX 0.0
IY 0.0
XY 0.0
YY 0.0
ZY 0.0
IZ 0.0
XZ 0.0
YZ 0.0
ZZ 0.0


In [ ]:
# raw_fidelity_list = eab_result["fidelity_list"]


# # sys.exit(0)

# fidelity_list = {}
# stdev_list = {}

# for pauli_label in pauli_request_list:
#     if(pauli_label == 'I'*nqubit):
#         fidelity_list[pauli_label] = 1.0
#         stdev_list[pauli_label] = 0.0
#     else:
#         alpha, alpha_err = EAB_process_modified.fit_EAB(depth, raw_fidelity_list[pauli_label])
#         fidelity_list[pauli_label] = alpha
#         stdev_list[pauli_label] = alpha_err

# # print(fidelity_list)

# # print(stdev_list)
# print("Parameters: n = %d, C = %d, " % (nqubit,C), "L = ", str(depth))
# use_density_matrix=False
# use_readout_error =False
# shots=2000
# if use_density_matrix:
#     print("Density matrix based simulation")
# else:
#     print("Measurement based simulation, shots = %d" % shots)

# if use_readout_error is True:
#     print("Measurement bitflip rate = %f" % eps_readout)
# else:
#     print("No readout error")

# # Average fidelity
# print("Total error = ", 1-np.mean(list(fidelity_list.values())))

# print("Label / Pauli infidelity / Standard deviation")
# for pauli_label in pauli_request_list:
#     print(pauli_label[::-1], 1-fidelity_list[pauli_label], stdev_list[pauli_label])
# # print('Effective noise rate = ' + str(1-np.average(list(fidelity_list.values()))))



In [ ]:
# True value for this specific noise model
eps=0
def f_true(P):
    ans = 0.0
    for Pi in P:
        if Pi == 'Z' or Pi == 'Y':
            ans += 2*eps
    return ans
def p_true(P):
    if P == "XI" or P == "IX":
        return eps
    else:
        return 0.0

print("Label / True infidelity/ True Pauli error")
for P in pauli_request_list:
    print(P[::-1]," ",f_true(P)," ",p_true(P))

In [ ]:
print (raw_fidelity_list)

In [ ]:
label="-XY"
print (label[-2:])

In [ ]:
p